# Debugger and Profiler

In [4]:
# This is my dry run script, to test the entrypoint in isolation
! python3 pytorch_cifar_profiling.py --gpu False

[2022-01-10 15:49:58.939 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:510 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-01-10 15:49:59.407 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:510 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
batch_size:128
epoch:1
gpu:False
model:resnet50
hook:  None
Files already downloaded and verified
Files already downloaded and verified
START TRAINING
START VALIDATING
Epoch 0: train loss 1258.350, val loss 294.829, in 1740.6 sec
Median training time per Epoch=1740.6 sec


In [2]:
# install dependencies
!pip install smdebug

In [3]:
hyperparameters = {
    "batch_size": 2048,
    "gpu": True,
    "epoch": 2,
    "model": "resnet50",
}

In [4]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),    
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

In [5]:
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

#TODO: Can you create the profiler and debugger configs
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(num_steps=10)
)
debugger_config = DebuggerHookConfig(
    hook_parameters={
        'train.save_internal': '100',
        'eval.save_internal': '10'
    }
)

In [6]:
import sagemaker
from sagemaker.pytorch import PyTorch

#TODO: Create the estimator to train your model
estimator = PyTorch(
  base_job_name='sm-debugger-profiler-ex-cifar-job',
  role=sagemaker.get_execution_role(),
  #instance_type='ml.m5.large',
  instance_type='ml.p3.2xlarge',
  instance_count=2,
  entry_point='./pytorch_cifar_profiling.py',
  framework_version='1.8',
  py_version='py36',
  hyperparameters=hyperparameters,  
  rules=rules,
  debugger_hook_config=debugger_config,
  profiler_config=profiler_config  
)

In [7]:
estimator.fit(wait=True)

2022-01-11 13:46:37 Starting - Starting the training job...
2022-01-11 13:47:02 Starting - Launching requested ML instancesVanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LossNotDecreasing: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2022-01-11 13:48:03 Starting - Preparing the instances for training.........
2022-01-11 13:49:34 Downloading - Downloading input data...
2022-01-11 13:50:03 Training - Downloading the training image.................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-11 13:52:51,828 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-11 13:52:51,850 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.

2022-01-11 13:53:05 Training - Training image download completed. Training in progress.bash: cannot set terminal pr

In [8]:
import boto3

session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
# training_job_name = 

print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

Training jobname: sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196
Region: us-east-1


In [9]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

[2022-01-11 14:08:54.430 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:38 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-152200895494/sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/debug-output


In [10]:
# TODO: Can you print the names of all the tensors that were tracked
# TODO: Can you print the number of datapoints for one of those tensors
# for both train and eval mode
trial.tensor_names()

[2022-01-11 14:08:54.801 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:38 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2022-01-11 14:08:55.821 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:38 INFO trial.py:210] Loaded all steps


['CrossEntropyLoss_output_0',
 'gradient/ResNet_bn1.bias',
 'gradient/ResNet_bn1.weight',
 'gradient/ResNet_conv1.weight',
 'gradient/ResNet_fc.bias',
 'gradient/ResNet_fc.weight',
 'gradient/ResNet_layer1.0.bn1.bias',
 'gradient/ResNet_layer1.0.bn1.weight',
 'gradient/ResNet_layer1.0.bn2.bias',
 'gradient/ResNet_layer1.0.bn2.weight',
 'gradient/ResNet_layer1.0.bn3.bias',
 'gradient/ResNet_layer1.0.bn3.weight',
 'gradient/ResNet_layer1.0.conv1.weight',
 'gradient/ResNet_layer1.0.conv2.weight',
 'gradient/ResNet_layer1.0.conv3.weight',
 'gradient/ResNet_layer1.0.downsample.0.weight',
 'gradient/ResNet_layer1.0.downsample.1.bias',
 'gradient/ResNet_layer1.0.downsample.1.weight',
 'gradient/ResNet_layer1.1.bn1.bias',
 'gradient/ResNet_layer1.1.bn1.weight',
 'gradient/ResNet_layer1.1.bn2.bias',
 'gradient/ResNet_layer1.1.bn2.weight',
 'gradient/ResNet_layer1.1.bn3.bias',
 'gradient/ResNet_layer1.1.bn3.weight',
 'gradient/ResNet_layer1.1.conv1.weight',
 'gradient/ResNet_layer1.1.conv2.weigh

In [11]:
train_datapoints = len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN))
eval_datapoints = len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL))
print(train_datapoints, eval_datapoints)

1 1


In [12]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-152200895494/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 10, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 10, }'}}
s3 path:s3://sagemaker-us-east-1-152200895494/sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/profiler-output


Profiler data from system is available


In [13]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

[2022-01-11 14:26:30.532 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:38 INFO metrics_reader_base.py:134] Getting 40 event files
select events:['total']
select dimensions:['CPU', 'GPU']
filtered_events:{'total'}
filtered_dimensions:{'CPUUtilization-nodeid:algo-2', 'GPUMemoryUtilization-nodeid:algo-1', 'GPUUtilization-nodeid:algo-1', 'GPUUtilization-nodeid:algo-2', 'GPUMemoryUtilization-nodeid:algo-2', 'CPUUtilization-nodeid:algo-1'}


In [14]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

You will find the profiler report in s3://sagemaker-us-east-1-152200895494/sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output


In [15]:
! aws s3 ls {rule_output_path} --recursive

2022-01-11 14:09:02     449979 sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-report.html
2022-01-11 14:09:01     312375 sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2022-01-11 14:08:56        905 sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2022-01-11 14:08:56      62829 sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2022-01-11 14:08:56       1898 sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2022-01-11 14:08:56        130 sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2022-01-11 14:08:56     

In [16]:
! aws s3 cp {rule_output_path} ./ --recursive

download: s3://sagemaker-us-east-1-152200895494/sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json to ProfilerReport/profiler-output/profiler-reports/Dataloader.json
download: s3://sagemaker-us-east-1-152200895494/sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json to ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
download: s3://sagemaker-us-east-1-152200895494/sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json to ProfilerReport/profiler-output/profiler-reports/BatchSize.json
download: s3://sagemaker-us-east-1-152200895494/sm-debugger-profiler-ex-cifar-job-2022-01-11-13-46-36-196/rule-output/ProfilerReport/profiler-output/profiler-reports/IOBottleneck.json to ProfilerReport/profiler-output/profiler-reports/IOBott

In [17]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [18]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")